<h1><center>Stroke Prediction</center></h1>
<center>September 2024</center>
<center>Celine Ng</center>

# Table of Contents

1. Project Introduction   
    1. Libraries Installation
    1. Dataset
    1. Data Cleaning
        1. Duplicate rows
        2. Datatypes
       3. Unique values
       4. Missing values
       5. Insights
1. EDA
    1. Distribution
        1. Numeric Data
        2. Categorical Data
        3. Label
            1. Summary
    1. Correlation
        1. Correlation with numerical features
        1. Multicollinearity - Variance Inflation Factor
    1. Statistic Inference
        1. Target Population
        2. Transform 'AnnualIncome'
        3. Hypothesis Testing
1. Preprocessing
    1. Transformations
    1. Data Splitting
1. Models
    1. Apply various models
    1. Ensemble - Voting
1. Improvements

# 1. Project Introduction

## 1. Libraries Installation

In [163]:
from sklearn.preprocessing import LabelEncoder
%%capture
%pip install -r requirements.txt

In [164]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from utils.eda import *

from sklearn.compose import make_column_selector as selector

custom_params = {
    "axes.spines.right": False,
    "axes.spines.top": False,
    "figure.figsize": (8, 5),
}
sns.set_theme(
    context="notebook", style="whitegrid", rc=custom_params
)
px.defaults.template = "seaborn"
pd.set_option("display.precision", 2)

%matplotlib inline

## 2. Dataset

Objective: Brief overview of our dataset, including the features and label

The dataset was downloaded from Kaggle, [Stroke_Prediction_Data](https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset)
,<br>
on 10 September 2024. It will be used to predict whether a patient is likely
 <br> to get a stroke. <br>
The data contains 11 clinical features, like gender, age, smoking status, <br>
etc, that help describe each patient.

Features: <br>
1) id: unique identifier
2) gender: "Male", "Female" or "Other"
3) age: age of the patient
4) hypertension: 0 if the patient doesn't have hypertension, 1 if the patient has hypertension
5) heart_disease: 0 if the patient doesn't have any heart diseases, 1 if the patient has a heart disease
6) ever_married: "No" or "Yes"
7) work_type: "children", "Govt_jov", "Never_worked", "Private" or "Self-employed"
8) Residence_type: "Rural" or "Urban"
9) avg_glucose_level: average glucose level in blood
10) bmi: body mass index
11) smoking_status: "formerly smoked", "never smoked", "smokes" or "Unknown"*
<br>

Label: <br>
stroke: 1 if the patient had a stroke or 0 if not <br>

In [165]:
stroke_data = pd.read_csv('data/healthcare-dataset-stroke-data.csv')
display(stroke_data.head())
shape = stroke_data.shape
print(f'Number of rows: {shape[0]}\nNumber of columns: {shape[1]}')

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


Number of rows: 5110
Number of columns: 12


## 3. Data cleaning
Objective:
1. Closer look at the values that consist of our data
2. Look out for duplicates, and missing and/or unusual values

### 1. Duplicate rows

In [166]:
print(stroke_data.id.duplicated().any())

False


After confirming there is no duplicated id number/cases, since id number 
<br>should not be relevant information to base our prediction on, we can <br>
remove it.

In [167]:
stroke_data = stroke_data.drop(columns=['id'])

### 2. Datatypes

In [168]:
datatype_data = stroke_data.dtypes
datatype_data

gender                object
age                  float64
hypertension           int64
heart_disease          int64
ever_married          object
work_type             object
Residence_type        object
avg_glucose_level    float64
bmi                  float64
smoking_status        object
stroke                 int64
dtype: object

Certain features have unexpected datatypes, like age, hypertension, and <br>
heart_disease. Looking into the values itself will help clarify data types.

### 3. Unique values

In [169]:
stroke_data.nunique()

gender                  3
age                   104
hypertension            2
heart_disease           2
ever_married            2
work_type               5
Residence_type          2
avg_glucose_level    3979
bmi                   418
smoking_status          4
stroke                  2
dtype: int64

To understand why some features have unexpected amount of unique values, <br>
display the unique values and its count for all features.

In [170]:
for column in stroke_data.columns:
    df = stroke_data[column].value_counts()
    display(df)

gender
Female    2994
Male      2115
Other        1
Name: count, dtype: int64

age
78.00    102
57.00     95
52.00     90
54.00     87
51.00     86
        ... 
1.40       3
0.48       3
0.16       3
0.40       2
0.08       2
Name: count, Length: 104, dtype: int64

hypertension
0    4612
1     498
Name: count, dtype: int64

heart_disease
0    4834
1     276
Name: count, dtype: int64

ever_married
Yes    3353
No     1757
Name: count, dtype: int64

...

Residence_type
Urban    2596
Rural    2514
Name: count, dtype: int64

avg_glucose_level
93.88     6
91.68     5
91.85     5
83.16     5
73.00     5
         ..
111.93    1
94.40     1
95.57     1
66.29     1
85.28     1
Name: count, Length: 3979, dtype: int64

bmi
28.7    41
28.4    38
26.7    37
27.6    37
26.1    37
        ..
48.7     1
49.2     1
51.0     1
49.4     1
14.9     1
Name: count, Length: 418, dtype: int64

smoking_status
never smoked       1892
Unknown            1544
formerly smoked     885
smokes              789
Name: count, dtype: int64

stroke
0    4861
1     249
Name: count, dtype: int64

Note that categorical features need to be preprocessed for consistency <br>
before applying learning model, as their datatypes are not the same. <br> <br>
Regarding age, its datatype is float due to registering young children's age
 <br>
more precisely.<br><br>
In general, the datasets is imbalanced as out of 5110, only 249 responded <br>
positively to strokes.

In [171]:
under18_children = stroke_data[(stroke_data.age<18) & (stroke_data
                                              .work_type=='children')].shape[0]
total_children = stroke_data[stroke_data.work_type == 'children'].shape[0]
print(f"{under18_children/total_children*100}% of work_type registered as "
      f"'children' are under 18years old.")

100.0% of work_type registered as 'children' are under 18years old.


We might have found redundant information between cases typed as 'children' 
<br>
in the work_type feature, and cases under 18 years old and 'Never worked'. 

### 4. Missing values

In [172]:
missing_values(df=stroke_data)

,features,null_count,null_%
0,gender,0,0.000000
1,age,0,0.000000
2,hypertension,0,0.000000
3,heart_disease,0,0.000000
4,ever_married,0,0.000000
5,work_type,0,0.000000
6,Residence_type,0,0.000000
7,avg_glucose_level,0,0.000000
8,bmi,201,3.933464
9,smoking_status,0,0.000000


To properly impute missing values, it is necessary to first understand the <br>
sampling population and if the missing values belong to a specific subset of
 <br>
sampling population.

### 5. Insights

1. There are no duplicated rows, and each row is identified by a unique id.
2. Regarding unique values from each feature, categorical columns do not <br>
show a consistent labeling, preprocessing before applying learning model will 
<br>
be necessary. 
3. The dataset is imbalanced, requires special attention when handling data 
<br>
before applying model.
4. Possible redundant information between features 'age' and 'work_type', <br>
for under 18 years olds and registered as children for their 'work_type'.
5. All missing values come from feature 'bmi', needs further analysis for <br>
proper imputation.

# 2. EDA

1. Data distribution (in percentage)
2. Comparison and A/B test between data with and without stroke 
3. Check and handle outliers, missing values

create plots for missing values

## 1. Distribution

Identify features, label, and different feature types.

In [178]:
feature_columns = stroke_data.columns[:-1]
label_column = stroke_data.columns[-1]

In [179]:
categorical_columns_selector = selector(dtype_exclude='float64')
numerical_columns_selector = selector(dtype_include='float64')

categorical_columns = categorical_columns_selector(stroke_data)
numerical_columns = numerical_columns_selector(stroke_data)

### 1. Numerical data 

### 2. Categorical data

### 3. Label